In [4]:
import xml.etree.cElementTree as ET
import pprint

#audit way tags for unexpected city names in Austin map

osmfile = "sample.osm"

expected = ("Austin", "Pflugerville", "Round Rock")

def is_city_name(elem):
    return (elem.attrib['k'] =="addr:city")

def audit_city(osmfile):
     city_file = open(osmfile, "r")
     for event, elem in ET.iterparse(city_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'addr:city':
                    city = tag.attrib['v'].strip()
                    if city not in expected: 
                        print city
                        
                         
     city_file.close()

audit_city("sample.osm")

#audit of sample file shows names which appear to be outside of what would be considered Austin

Manchaca
Cedar Park, TX
Buda
Sunset Valley
Buda
Lakeway
Sunset Valley
Cedar Park


In [2]:
import xml.etree.cElementTree as ET
import pprint
import re

#audit way tags for unexpected ZIP CODES in Austin map

osmfile = "sample.osm"

POSTCODE = re.compile(r'[A-z]\d[A-z]\s?\d[A-z]\d')
def audit_postcode(osmfile):
    post_file = open(osmfile, "r")
    for event, elem in ET.iterparse(post_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'addr:postcode':
                    post_code = re.sub(" ", "", tag.attrib['v'].strip())
                    m = POSTCODE.match(post_code)
                    if m is None:
                        print post_code      
    post_file.close()

audit_postcode("sample.osm")


78701
78759
78757
78759
78652
78751
78751
78752
78704
78756
78746
78750
78613
78660
78735
78701
78745
TX78745
78746
78703
78751
78759
78704
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78752
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78731
78731
78731
78731
78731
78731
78731
78731
78757
78757
78731
78731
78731
78731
78757
78757
78751
78751
78751
78731
78751
78751
78751
78757
78757
78751
78731
78751
78751
78751
78751
78751
78759
78759
78759
78759
78759
78759
78759
78759
78759
78759
78751
78751
78723
78723
78723
78723
78723
78757
78757
78722
78722
78757
78757
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78723
78722
78757
78757
78757
78722
78722
78722
78

In [1]:
import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "austin.osm"  # Replace this with your osm file
SAMPLE_FILE = "test_sample.osm"

k = 125 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [1]:
"""
Your task is to use the iterative parsing to process the map file and
find out not only what tags are there, but also how many, to get the
feeling on how much of which data you can expect to have in the map.
Fill out the count_tags function. It should return a dictionary with the 
tag name as the key and number of times this tag can be encountered in 
the map as value.

Note that your code will be tested with a different data file than the 'example.osm'
"""
import xml.etree.cElementTree as ET
import pprint



def count_tags(filename):
        # YOUR CODE HERE
    tagDict = {}
    parser = ET.iterparse(filename)
    for __, elem in parser:
        if elem.tag in tagDict:
            tagDict[elem.tag]+=1
        else:
            tagDict[elem.tag] = 1
        elem.clear()
    del parser
    return tagDict
    
tags = count_tags('austin.osm')
pprint.pprint(tags)
    


{'bounds': 1,
 'member': 18810,
 'nd': 5892639,
 'node': 5358645,
 'osm': 1,
 'relation': 2161,
 'tag': 2022657,
 'way': 564388}


In [5]:
import xml.etree.cElementTree as ET
import pprint
import re
"""
Your task is to explore the data a bit more.
Before you process the data and add it into your database, you should check the
"k" value for each "<tag>" and see if there are any potential problems.

We have provided you with 3 regular expressions to check for certain patterns
in the tags. As we saw in the quiz earlier, we would like to change the data
model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, we have to see if we have such tags, and if we have any tags with
problematic characters.

Please complete the function 'key_type', such that we have a count of each of
four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
See the 'process_map' and 'test' functions for examples of the expected format.
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        key = element.attrib['k']
        if lower.match(key):
            keys['lower'] = keys['lower'] + 1
        elif lower_colon.match(key):
            keys["lower_colon"] = keys["lower_colon"] + 1
        elif problemchars.match(key):
            keys["problemchars"] = keys["problemchars"] + 1
        else:
            keys["other"] = keys["other"] + 1
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map('sample.osm')
pprint.pprint(keys)


{'lower': 56178, 'lower_colon': 44554, 'other': 371, 'problemchars': 0}


In [5]:
import xml.etree.cElementTree as ET
import pprint
import re
"""
Your task is to explore the data a bit more.
The first task is a fun one - find out how many unique users
have contributed to the map in this particular area!

The function process_map should return a set of unique user IDs ("uid")
"""

def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('uid'):
            try:
                users.add(element.get('uid'))
            except:
                pass

    return users 

users = process_map('sample.osm')
pprint.pprint(users)


set(['1007528',
     '1012362',
     '103253',
     '1044834',
     '104962',
     '105002',
     '1051550',
     '105454',
     '1058308',
     '105839',
     '1058397',
     '1058666',
     '105946',
     '106663',
     '1072516',
     '1073361',
     '107681',
     '108775',
     '109362',
     '110263',
     '110639',
     '110797',
     '1110270',
     '1132286',
     '113375',
     '113450',
     '11566',
     '115918',
     '1171541',
     '118134',
     '119002',
     '1195151',
     '119748',
     '119881',
     '121241',
     '1213926',
     '1219875',
     '1227690',
     '1240849',
     '12434',
     '124836',
     '1260280',
     '128470',
     '12919',
     '129841',
     '129867',
     '130472',
     '1306',
     '131059',
     '13203',
     '132444',
     '1326818',
     '135807',
     '1363265',
     '136520',
     '1376118',
     '139731',
     '142426',
     '142739',
     '1429602',
     '143162',
     '1434650',
     '143480',
     '143523',
     '1443840',
     '1

In [2]:
#Reviewing street names to create set of names to change

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

osm_file = open("sample.osm", "r")
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))

if __name__ == '__main__':
    audit("sample.osm")


{'101': set(['4207 James Casey st #101']),
 '1431': set(['Farm-to-Market Road 1431', 'Old Farm-to-Market 1431']),
 '1625': set(['Farm-to-Market Road 1625']),
 '1626': set(['Farm-to-Market Road 1626']),
 '2222': set(['Ranch to Market Road 2222']),
 '2769': set(['Farm-to-Market Road 2769']),
 '290': set(['East Highway 290', 'US Highway 290', 'West US Highway 290']),
 '35': set(['North Interstate Highway 35']),
 '620': set(['RM 620', 'Ranch Road 620', 'Ranch-to-Market Road 620']),
 '812': set(['Farm-to-Market Road 812']),
 '969': set(['FM 969']),
 '973': set(['Farm-to-Market Road 973']),
 'A': set(['Avenue A']),
 'Acres': set(['Green Acres']),
 'Affirmed': set(['Affirmed']),
 'Alley': set(['Oak Alley']),
 'Alto': set(['Camino Alto']),
 'Amistad': set(['Circulo de Amistad']),
 'B': set(['Avenue B']),
 'Barrhead': set(['Barrhead']),
 'Bend': set(['Badger Bend',
              'Bangor Bend',
              'Bantom Woods Bend',
              'Barrel Bend',
              'Basie Bend',
          

In [ ]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "austin.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def load_new_tag(element, secondary, default_tag_type):
    """
    Load a new tag dict to go into the list of dicts for way_tags, node_tags
    """
    new = {}
    new['id'] = element.attrib['id']
    if ":" not in secondary.attrib['k']:
        new['key'] = secondary.attrib['k']
        new['type'] = default_tag_type
    else:
        post_colon = secondary.attrib['k'].index(":") + 1
        new['key'] = secondary.attrib['k'][post_colon:]
        new['type'] = secondary.attrib['k'][:post_colon - 1]
    new['value'] = secondary.attrib['v']
    #print "!23123"
    #print secondary.attrib['v']
    #print"!2312"
    return new


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for attrib, value in element.attrib.iteritems():
            if attrib in node_attr_fields:
                node_attribs[attrib] = value
        
        # for elements within the top element
        for secondary in element.iter():
            if secondary.tag == 'tag':
                if problem_chars.match(secondary.attrib['k']) is not None:
                    continue
                else:
                    new = load_new_tag(element, secondary, default_tag_type)
                    tags.append(new)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for attrib, value in element.attrib.iteritems():
            if attrib in way_attr_fields:
                way_attribs[attrib] = value
                
        counter = 0
        for secondary in element.iter():
            if secondary.tag == 'tag':
                if problem_chars.match(secondary.attrib['k']) is not None:
                    continue
                else:
                    new = load_new_tag(element, secondary, default_tag_type)
                    tags.append(new)
            if secondary.tag == 'nd':
                newnd = {}
                newnd['id'] = element.attrib['id']
                newnd['node_id'] = secondary.attrib['ref']
                newnd['position'] = counter
                counter += 1
                way_nodes.append(newnd)
        
        #print {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is False:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


{'way_tags': [{'value': 'yes', 'type': 'regular', 'id': '4531226', 'key': 'toll'}, {'value': '3', 'type': 'regular', 'id': '4531226', 'key': 'layer'}, {'value': 'yes', 'type': 'regular', 'id': '4531226', 'key': 'oneway'}, {'value': 'motorway_link', 'type': 'regular', 'id': '4531226', 'key': 'highway'}], 'way_nodes': [{'position': 0, 'node_id': '28110335', 'id': '4531226'}, {'position': 1, 'node_id': '28110339', 'id': '4531226'}, {'position': 2, 'node_id': '1329101021', 'id': '4531226'}, {'position': 3, 'node_id': '1332607859', 'id': '4531226'}, {'position': 4, 'node_id': '1619623761', 'id': '4531226'}], 'way': {'changeset': '33813445', 'uid': '2929338', 'timestamp': '2015-09-05T12:03:25Z', 'version': '11', 'user': 'gabis_telenav', 'id': '4531226'}}
{'way_tags': [{'value': 'yes', 'type': 'regular', 'id': '4531257', 'key': 'toll'}, {'value': 'yes', 'type': 'regular', 'id': '4531257', 'key': 'oneway'}, {'value': 'motorway_link', 'type': 'regular', 'id': '4531257', 'key': 'highway'}, {'val